# Langchain Basic Orchestration

In [2]:
# Simple print output function for wrapping response
def jprint(content, wrap_at=90):
    """Prints wrapped text or renders Markdown in Jupyter."""
    try:
        from IPython.display import display, Markdown

        display(Markdown(content))
    except ImportError:
        import textwrap

        print(textwrap.fill(content, width=wrap_at))

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize a Chat Model
chat_model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = PromptTemplate.from_template("Summarize {topic} in one paragraph.")

# Initialize the parser
parser = StrOutputParser()

# Formatting the prompt with a variable
# formatted_prompt = prompt.format(topic="Memory in AI")

### Creating a Basic Chain
A Chain is a sequence of steps that process input one stage at a time. Each step transforms information before passing it forward

In [4]:
# Chains turn reasoning into a structured pipeline
# Here we combine: Prompt -> Model -> Output Parser
chain = prompt | chat_model | parser

response = chain.invoke({"topic": "Chains in LangChain"})
jprint(response)

Chains in LangChain are a feature that allow users to create a series of automated actions that are triggered by certain events or conditions. These chains can be customized to perform a variety of tasks such as sending notifications, updating records, and executing external scripts. Users can define the specific conditions and actions for each step in the chain, making it a powerful tool for automating workflows and improving efficiency in language-related tasks.

### Adding Memory
Memory stores information from previous interactions. This enables continuity and context-aware conversations

**NOTE - better to use langgraph now**

In [5]:
# NOTE: This is the old style, better to use Langgraph now!
# Change the import to use the classic path
from langchain_classic.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "My name is Gemini"}, {"output": "Hello Gemini!"})

print(memory.load_memory_variables({}))

{'history': 'Human: My name is Gemini\nAI: Hello Gemini!'}


/var/folders/k5/rnrtn2ld6fsg4r3135y8rph40000gn/T/ipykernel_31561/4135528638.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


### Tools and Agents
Tools enable LLMs to take real actions like API calls or database queries. An Agent uses reasoning to decide which tool to use and when.

In LangChain, an Agent acts as a reasoning engine. It doesn't just give one answer; it decides which Tool to use based on your question.

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import datetime

# 1. Initialize Model
model = ChatOpenAI(model="gpt-3.5-turbo")


# 2. Define the Tool
def get_current_time():
    return datetime.datetime.now().strftime("%I:%M %p")


# 3. Decision Logic Prompt
# We tell the AI it is disconnected from a live clock.
logic_prompt = PromptTemplate.from_template(
    "You are an assistant without access to a real-time clock. "
    "Tool Available: 'TimeCheck' (provides the current time). "
    "Question: {question} "
    "Do you need the tool to answer accurately? Answer 'Yes' or 'No'."
)

chain = logic_prompt | model | StrOutputParser()

# 4. The Question
question = "What time is it right now?"
decision = chain.invoke({"question": question})

print(f"Agent Decision: {decision}")

# 5. Conditional Tool Execution
if "Yes" in decision:
    result = get_current_time()
    print(f"Tool Output: The current time is {result}")
else:
    print("The AI thought it knew the time... but it's likely hallucinating!")

Agent Decision: Yes
Tool Output: The current time is 06:42 AM
